In [4]:
import requests
import datetime
import base64
from urllib.parse import urlencode

import json

import numpy as np

Класс SpotifyAPI необходим для осуществления непосредственно запросов информации о треках, исполнителях и т.д 

In [14]:
from creds import client_id
from creds import client_secret
# для получения токена - лежат в creds.py

class SpotifyAPI(object):
    __client_id = None
    __client_secret = None

    access_token = None

    # время когда токен перестанет быть активным
    expires = datetime.datetime.now()

    # мин к-во баллов, которое нужно песне, чтобы точно подходить под тип
    point_min = 60

    # типы, которые можно использовать в методе get()
    get_types = ['albums', 'tracks', 'playlists', 'audio-features']
    # параметры песен, которые используются для типизации
    chars = ["danceability", "energy", "loudness", "tempo"]
    # max и min значения параметров (какие вообще возможны) (всех, кроме темпа)
    MM_chars = [(0.0, 1.0), (0.0, 1.0), (-60, 0)]
    # для определения, для типа подходит высокий темп или нет
    type_tempo_high = {
        'before sleep': False,
        'for workout': True,
        'for a party': True}
    song_types = ['before sleep', 'for workout', 'for a party']

    type_params = {}

    GOOD_STATUS_CODE = 200

    def __init__(self, client_id, client_secret):
      self.__client_id = client_id
      self.__client_secret = client_secret


    def init_params(self):
      i = 0
      # тут мы считаем из файла посчитанные min, max, mean параметров
      # (ниже будет)
      for type_ in self.song_types:
        a = np.loadtxt(f'{type_} MMM.txt')
        self.type_params[type_] = a
        i += 1

    def client_creds(self):
        return f"{self.__client_id}:{self.__client_secret}"

    # получение токена
    def get_access_token(self):
        token_url = "https://accounts.spotify.com/api/token"
        token_data = {
            "grant_type": "client_credentials"
        }
        client_creds_b64 = base64.b64encode(self.client_creds().encode())
        token_headers = {
            "Authorization": f"Basic {client_creds_b64.decode()}"  
            # <base64 encoded client_id:client_secret>
        }
        # запрашиваем токен
        r = requests.post(token_url, data=token_data, headers=token_headers)
        # проверка, что токен получен
        valid_request = r.status_code == self.GOOD_STATUS_CODE

        if valid_request:
            token_response_data = r.json()
            now = datetime.datetime.now()
            self.access_token = token_response_data['access_token']
            expires_in = token_response_data['expires_in']  # seconds
            self.expires = now + datetime.timedelta(seconds=expires_in)
            return True
        else:
            return False

    # проверка необходимости получения нового токена
    def refresh_token(self):
        expires = self.expires
        now = datetime.datetime.now()
        if not(self.access_token) or expires < now:
            self.get_access_token()

    # поиск трека, альбома или исполнителя по названию (имени)
    def search(self, artist_name, track_name, album_name, search_type, limit=20, offset=0):
        self.refresh_token()

        headers = {
            "Authorization": f"Bearer {self.access_token}"
        }
        endpoint = "https://api.spotify.com/v1/search"
        data = None
        if search_type == 'track':
            data = urlencode(
                {"q": "artist:" + artist_name + " track:" + track_name, 
                 "type": search_type, 
                 "limit": limit,
                 "offset": offset}, 
                 safe=":" )
        elif search_type == 'artist':
            data = urlencode({"q": artist_name, 
                              "type": search_type, 
                              "limit": limit, 
                              "offset": offset})
        else:
            data = urlencode({"q": album_name, 
                              "type": search_type, 
                              "limit": limit, 
                              "offset": offset})

        lookup_url = f"{endpoint}?{data}"
        r = requests.get(lookup_url, headers=headers)

        if (r.status_code != self.GOOD_STATUS_CODE):
            raise Exception(r.json()['error']['message'])

        return r.json()

    # получение информации о треке, плейлисте и т.д по его id
    def get(self, type_, id):
        self.refresh_token()
        headers = {
            "Authorization": f"Bearer {self.access_token}"
        }
        lookup_url = f"https://api.spotify.com/v1/{type_}/{id}"
        r = requests.get(lookup_url, headers=headers)

        if (r.status_code != self.GOOD_STATUS_CODE):
            raise Exception((r.json()['error']['message'], r.status_code))
        return r.json()

    # получение информации по url конечной точки
    def get_by_url(self, lookup_url):
        self.refresh_token()
        headers = {
            "Authorization": f"Bearer {self.access_token}"
        }
        r = requests.get(lookup_url, headers=headers)

        if (r.status_code != self.GOOD_STATUS_CODE):
            raise Exception((r.json()['error']['message'], r.status_code))
        return r.json()

    # проверка соответствия песни выбранному типу
    def check_track(self, track_id, type_):
        try:
          track_params = self.get('audio-features', track_id)
        except Exception:
          raise


        points = 0
        for i in range(len(self.chars) - 1):
            if self.type_params[type_][2][i] <= (self.MM_chars[i][1] + self.MM_chars[i][0]) / 2:
                if track_params[self.chars[i]] <= self.type_params[type_][2][i]:
                    points += 25
                elif track_params[self.chars[i]] <= self.type_params[type_][1][i]:
                    points += 12.5
                else:
                    points += 0
            else:
                if track_params[self.chars[i]] >= self.type_params[type_][2][i]:
                    points += 25
                elif track_params[self.chars[i]] >= self.type_params[type_][0][i]:
                    points += 12.5
                else:
                    points += 0

        if (self.type_tempo_high[type_]):
            if track_params['tempo'] >= self.type_params[type_][2][len(self.chars) - 1]:
                points += 25
            elif track_params['tempo'] >= self.type_params[type_][0][len(self.chars) - 1]:
                points += 12.5
        else:
            if track_params['tempo'] <= self.type_params[type_][2][len(self.chars) - 1]:
                points += 25
            elif track_params['tempo'] <= self.type_params[type_][1][len(self.chars) - 1]:
                points += 12.5

        print('Points :', points)
        if (points >= self.point_min):
            return (True, 1)
        elif (points >= self.point_min / 2):
            return (True, 0)
        else:
            return (False, 0)



client = SpotifyAPI(client_id=client_id, client_secret=client_secret)

Класс SongClassifier отвечает непосредственно за проверку соответствия песни выбранному типу, а также с помощью него происходил поиск значений параметров для каждого типа

In [15]:
import matplotlib.pyplot as plt
import numpy as np
import time

class SongClassifier(object):
  # типы, которые мы проверяем
  song_types = ['before sleep', 'for workout', 'for a party']
  # id плейлистов с достаточно большим количеством песен, каждый определенного типа
  song_types_examples = {
      song_types[0] : '37i9dQZF1DWZd79rJ6a7lp', 
      # https://open.spotify.com/playlist/37i9dQZF1DWZd79rJ6a7lp - плейлист Sleep 
      song_types[1] : '37i9dQZF1DX70RN3TfWWJh',
      # https://open.spotify.com/playlist/37i9dQZF1DX70RN3TfWWJh - плейлист Workout
      song_types[2] : '5xS3Gi0fA3Uo6RScucyct6'
      # https://open.spotify.com/playlist/5xS3Gi0fA3Uo6RScucyct6 - плейлист Party Songs
  }

  #параметры песен, которые интересны нам для анализа
  chars = ["danceability", "energy", "loudness", "tempo"]

  def __init__(self):
    return

  # по плейлисту возвращает список значений параметров  из track_params для каждой песни
  def analyse (self, playlist_id):
    list_char = []

    try:
      r = client.get('playlists', playlist_id)
    except Exception:
      raise

    url = r['tracks']['href']

    counter = 0
    while url:
      try:
        r = client.get_by_url(url)
      except Exception:
        raise

      for obj in r['items']:
        time.sleep(0.25)
        track_id = obj['track']['id']
        counter += 1
        print(counter)
      
        #обнаружилось, что не на все песни есть полный анализ
        try:
          track_info = client.get('audio-features', track_id)
        except Exception:
          continue
        
        list_char.append([track_info[elem] for elem in self.chars])

      url = r['next'] 

    return np.array(list_char)


cs = SongClassifier()

In [17]:
for song_type in cs.song_types:
  print("Analysing song_type =", song_type)
  list_char = cs.analyse(cs.song_types_examples[song_type])
  np.savetxt(f'{song_type} parameters.txt', list_char)

#тут вытаскивались параметры песен плейлистов, которые будут определять критерии попадания в тип

Analysing song_type = before sleep
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
Analysing song_type = for workout
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
Analysing song_type = for a party
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
2

In [18]:
for type_ in cs.song_types:
  a = np.loadtxt(f'{type_} parameters.txt')
  params = [a.min(axis=0), a.max(axis=0), a.mean(axis=0)]
  np.savetxt(f'{type_} MMM.txt', params)

#для каждого типа для каждого параметра сохранили минимальное, максимальное и среднее значения параметров

Как проверяется соответствие песни? 

Проверяется каждый параметр из списка для данного типа

Если mean значение параметра ДЛЯ ТИПА меньше половины:
--если параметр ПЕСНИ лежит до среднего значения - +25 очков

--если параметр ПЕСНИ между средним и максимальным - +12.5 очков

--иначе 0

Если mean значение параметра больше половины:

--если параметр песни лежит до среднего значения - +25 очков

--если параметр песни между средним и минимальным - +12.5 очков

--иначе 0

Если суммарно песня набирает хотя бы 30 очков из 100 - песня соответствует выбранному типу (если 60, то песня правда подходит, если 30 - песня относится к "сомнительной")

In [20]:
#проверяем пару песен 

client.init_params()

In [24]:
playlist_id = '2hstKFT7GOo34ZyFbMTXWN' 
#https://open.spotify.com/album/2hstKFT7GOo34ZyFbMTXWN
#Альбом с музыкой, которая поможет заснуть

r = client.get('albums', playlist_id)

url = r['tracks']['href']

counter = 0
total = 0
while url:
  r = client.get_by_url(url)
  for obj in r['items']:
    total += 1
    time.sleep(0.25)
    track_id = obj['id']
    print(f'Track {counter}')

    try:
      ans = client.check_track(track_id, 'before sleep')
      if ans[0]:
        counter += 1
        if ans[1]:
          print("YES!")
        else:
          print('Not sure...')
      else:
        print('NO!')
    except Exception:
      continue
  url = r['next'] 

print(counter / total * 100, 'процентов песен подходит под тип before sleep')

Track 0
Points : 87.5
YES!
Track 1
Points : 100
YES!
Track 2
Points : 100
YES!
Track 3
Points : 100
YES!
Track 4
Points : 100
YES!
Track 5
Points : 100
YES!
Track 6
Points : 100
YES!
Track 7
Points : 100
YES!
Track 8
Points : 87.5
YES!
Track 9
Points : 87.5
YES!
Track 10
Points : 87.5
YES!
Track 11
Points : 87.5
YES!
Track 12
Points : 87.5
YES!
Track 13
Points : 87.5
YES!
Track 14
Points : 100
YES!
Track 15
Points : 87.5
YES!
Track 16
Points : 87.5
YES!
Track 17
Points : 100
YES!
Track 18
Points : 100
YES!
Track 19
Points : 87.5
YES!
Track 20
Points : 100
YES!
Track 21
Points : 100
YES!
Track 22
Points : 87.5
YES!
Track 23
Points : 100
YES!
Track 24
Points : 100
YES!
Track 25
Points : 87.5
YES!
Track 26
Points : 87.5
YES!
Track 27
Points : 87.5
YES!
Track 28
Points : 87.5
YES!
Track 29
Points : 87.5
YES!
100.0 процентов песен подходит под тип before sleep


In [26]:
playlist_id = '37i9dQZF1DX70RN3TfWWJh' 
##https://open.spotify.com/playlist/37i9dQZF1DX70RN3TfWWJh - плейлист Workout
#По этому плейлисту считались параметры для for workout
#Проверка на корректность - неподходящая музыка правда отделяется :)

r = client.get('playlists', playlist_id)

url = r['tracks']['href']

counter = 0
total = 0
while url:
  r = client.get_by_url(url)

  for obj in r['items']:
    total += 1
    time.sleep(0.25)
    track_id = obj['track']['id']

    try:
      ans = client.check_track(track_id, 'before sleep')
      if ans[0]:
        counter += 1
        if ans[1]:
          print("YES!")
        else:
          print('Not sure...')
      else:
        print('NO!')
    except Exception:
      continue
  url = r['next'] 

print(counter / total * 100 , 'процентов песен подходит под тип before sleep')

Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 0
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 25
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
NO!
Points : 12.5
N